In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xml.etree.cElementTree as ET
import glob
import os
import json
import random
import shutil

from PIL import Image, ImageOps
def xml_to_yolo_bbox(bbox, w, h):
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h

    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h

    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h):
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2

    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)

    return [xmin, ymin, xmax, ymax]

classes = []

input_dir = "./annotations"
output_dir = "./labels"
image_dir = "./images"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

import glob

files = glob.glob(os.path.join(input_dir, "*.xml"))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall("object"):
        label = obj.find("name").text

        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)

        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding = "utf-8") as f:
            f.write("\n".join(result))

with open("./classes.txt", "w", encoding = "utf-8") as f:
    f.write(json.dumps(classes))

GGGG

In [4]:
!pip install ultralytics 


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [12]:
nc =  3
names = [
        "without_mask",
        "with_mask",
        "mask_weared_incorrect"
]

with open("data.yaml", "w") as f:
    f.write(f"train: /home/andrey/projects/ai_project/train/images\nval: /home/andrey/projects/ai_project/valid/images\n\nnc: {nc}\nnames: {names}")

In [13]:
!cat data.yaml

train: /home/andrey/projects/ai_project/train/images
val: /home/andrey/projects/ai_project/valid/images

nc: 3
names: ['without_mask', 'with_mask', 'mask_weared_incorrect']

In [7]:
import os
import random
import shutil
import yaml

DATASET_DIR = '.'
SAVE_SPLIT_DIR = './'

SIZE_VALID = 0.2

In [8]:
path_train = os.path.join(SAVE_SPLIT_DIR, 'train')
os.mkdir(path_train)
path_train_images = os.path.join(path_train, 'images')
os.mkdir(path_train_images)
path_train_labels = os.path.join(path_train, 'labels')
os.mkdir(path_train_labels)

path_valid = os.path.join(SAVE_SPLIT_DIR, 'valid')
os.mkdir(path_valid)
path_valid_images = os.path.join(path_valid, 'images')
os.mkdir(path_valid_images)
path_valid_labels = os.path.join(path_valid, 'labels')
os.mkdir(path_valid_labels)

In [9]:
path_images = os.path.join(DATASET_DIR, 'images')
path_labels = os.path.join(DATASET_DIR, 'labels')

In [10]:
images = os.listdir(path_images)

valid_images = random.sample(images, int(len(images) * SIZE_VALID))
print(f'Size valid = {len(valid_images)}')
for filename in images:
    only_filename = filename[:-4]

    if filename in valid_images:
        shutil.copyfile(
            os.path.join(path_images, filename),
            os.path.join(path_valid_images, filename),
        )
        shutil.copyfile(
            os.path.join(path_labels, f'{only_filename}.txt'),
            os.path.join(path_valid_labels, f'{only_filename}.txt'),
        )
    else:
        shutil.copyfile(
            os.path.join(path_images, filename),
            os.path.join(path_train_images, filename),
        )
        shutil.copyfile(
            os.path.join(path_labels, f'{only_filename}.txt'),
            os.path.join(path_train_labels, f'{only_filename}.txt'),
        )

Size valid = 170


In [15]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 11.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from ultralytics import settings
settings.update({"wandb": True})

In [2]:
import wandb

# Initialize your Weights & Biases environment
wandb.login(key=WANDB_API_KEY, force=True)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/andrey/.netrc
wandb: Currently logged in as: aerrries (aerrries-hse) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
import torch
import ultralytics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ultralytics.YOLO('yolo11n.yaml').to(device)

res = model.train(data='data.yaml', pretrained=True, epochs=200, batch=16, cache=False, project="mask_detection", name="yolo11n")

New https://pypi.org/project/ultralytics/8.3.94 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11n.yaml, data=data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=mask_detection, name=yolo11n8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


train: Scanning /home/andrey/projects/ai_project/train/labels.cache... 683 images, 0 backgrounds, 0 corrupt: 100%|██████████| 683/683 [00:00<?, ?it/s]
val: Scanning /home/andrey/projects/ai_project/valid/labels.cache... 170 images, 0 backgrounds, 0 corrupt: 100%|██████████| 170/170 [00:00<?, ?it/s]


Plotting labels to mask_detection/yolo11n8/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to mask_detection/yolo11n8
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


libpng warning: iCCP: Not recognizing known sRGB profile that has been edited:  30%|███       | 13/43 [00:01<00:03,  9.77it/s]
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited:  86%|████████▌ | 37/43 [00:03<00:00, 10.87it/s]
      1/200      2.77G       4.06      4.976      4.146         55        640: 100%|██████████| 43/43 [00:04<00:00,  9.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:00<00:00, 10.40it/s]

...

                   all        170        779      0.835      0.702      0.755      0.505



200 epochs completed in 0.251 hours.
Optimizer stripped from mask_detection/yolo11n8/weights/last.pt, 5.5MB
Optimizer stripped from mask_detection/yolo11n8/weights/best.pt, 5.5MB

Validating mask_detection/yolo11n8/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7930MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  5.38it/s]


                   all        170        779      0.755      0.757       0.78      0.516
             with_mask        157        640      0.899      0.897      0.937      0.625
          without_mask         54        116      0.746      0.733      0.741      0.463
 mask_weared_incorrect         19         23      0.621      0.642      0.661       0.46
Speed: 0.1ms preprocess, 3.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to mask_detection/yolo11n8


lr/pg0,▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▁▁▁▁
lr/pg1,█████▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
lr/pg2,▃███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▅▅▅▆▆▇▇▇▇▇█▇███████████████████████████
metrics/mAP50-95(B),▁▂▂▃▃▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇██▇▇████████████████
metrics/precision(B),▁▇█▇▆▆▆▇▇▇▆▇▇▇▇▇█▇▇▇▇▇▇██▇▇▇██▇███▇████▇
metrics/recall(B),▁▁▂▃▃▄▅▅▆▆▇▇▇▇█▇▇█▇▇▇▇▇▇▇▇█▇▇▇███▇█▇▇███
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▆▅▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁


In [4]:
import torch
import ultralytics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ultralytics.YOLO('yolo11s.yaml').to(device)

res = model.train(data='data.yaml', pretrained=True, epochs=200, batch=16, cache=False, project="mask_detection", name="yolo11s")

New https://pypi.org/project/ultralytics/8.3.94 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11s.yaml, data=data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=mask_detection, name=yolo11s3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


train: Scanning /home/andrey/projects/ai_project/train/labels.cache... 683 images, 0 backgrounds, 0 corrupt: 100%|██████████| 683/683 [00:00<?, ?it/s]
val: Scanning /home/andrey/projects/ai_project/valid/labels.cache... 170 images, 0 backgrounds, 0 corrupt: 100%|██████████| 170/170 [00:00<?, ?it/s]


Plotting labels to mask_detection/yolo11s3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to mask_detection/yolo11s3
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


libpng warning: iCCP: Not recognizing known sRGB profile that has been edited:  33%|███▎      | 14/43 [00:02<00:05,  5.66it/s]
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited:  86%|████████▌ | 37/43 [00:06<00:01,  5.87it/s]
      1/200      4.25G      4.077      4.734       4.15         55        640: 100%|██████████| 43/43 [00:07<00:00,  5.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:00<00:00,  9.11it/s]

...

                   all        170        779      0.812      0.737      0.799      0.534



200 epochs completed in 0.449 hours.
Optimizer stripped from mask_detection/yolo11s3/weights/last.pt, 19.2MB
Optimizer stripped from mask_detection/yolo11s3/weights/best.pt, 19.2MB

Validating mask_detection/yolo11s3/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7930MiB)
YOLO11s summary (fused): 100 layers, 9,413,961 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  4.72it/s]


                   all        170        779      0.811      0.785      0.813      0.557
             with_mask        157        640      0.912      0.895      0.944      0.653
          without_mask         54        116      0.781      0.724      0.791      0.506
 mask_weared_incorrect         19         23      0.739      0.737      0.705      0.514
Speed: 0.1ms preprocess, 4.6ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to mask_detection/yolo11s3


lr/pg0,███▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▁▁▁▁
lr/pg1,▃▆███▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁
lr/pg2,████▇▇▇▇▇▇▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▁▁▁▁
metrics/mAP50(B),▁▂▃▄▄▄▆▆▇▆▆▆▆▆▇▇▇▇▇▇▇▇█▇▇▇██▇▇█▇█▇█████▇
metrics/mAP50-95(B),▁▂▂▂▃▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████▇█████
metrics/precision(B),▁▄▇█▅▆▇▇▇▇▇▇▇▇██▇▆▇▇▇████▇▇▇▇███▇▇▇▇▇▇▇▇
metrics/recall(B),▁▁▄▄▆▆▅▆▆▇▇▇▇▇▇▇▇▇█▇▇▇█▇▇██▇██▇▇█▇██▇█▇█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,██▇▆▆▆▅▅▆▅▅▄▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▁▁▁


In [5]:
import torch
import ultralytics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ultralytics.YOLO('yolov8n.yaml').to(device)

res = model.train(data='data.yaml', pretrained=True, epochs=200, batch=16, cache=False, project="mask_detection", name="yolo8n")

New https://pypi.org/project/ultralytics/8.3.94 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=mask_detection, name=yolo8n4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


train: Scanning /home/andrey/projects/ai_project/train/labels.cache... 683 images, 0 backgrounds, 0 corrupt: 100%|██████████| 683/683 [00:00<?, ?it/s]
val: Scanning /home/andrey/projects/ai_project/valid/labels.cache... 170 images, 0 backgrounds, 0 corrupt: 100%|██████████| 170/170 [00:00<?, ?it/s]


Plotting labels to mask_detection/yolo8n4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to mask_detection/yolo8n4
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


libpng warning: iCCP: Not recognizing known sRGB profile that has been edited:  30%|███       | 13/43 [00:01<00:03,  9.46it/s]
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited:  86%|████████▌ | 37/43 [00:03<00:00, 10.71it/s]
      1/200      2.79G      4.097      4.822      4.164         55        640: 100%|██████████| 43/43 [00:04<00:00,  9.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:00<00:00, 12.25it/s]

...

                   all        170        779      0.852      0.699      0.786      0.522



200 epochs completed in 0.234 hours.
Optimizer stripped from mask_detection/yolo8n4/weights/last.pt, 6.3MB
Optimizer stripped from mask_detection/yolo8n4/weights/best.pt, 6.3MB

Validating mask_detection/yolo8n4/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7930MiB)
YOLOv8n summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  5.72it/s]


                   all        170        779      0.838      0.698      0.786      0.524
             with_mask        157        640      0.972      0.848      0.936      0.643
          without_mask         54        116      0.831      0.638      0.749      0.468
 mask_weared_incorrect         19         23      0.711      0.609      0.672      0.462
Speed: 0.1ms preprocess, 3.5ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to mask_detection/yolo8n4


lr/pg0,███▇▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg1,▆████▇▇▆▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁
lr/pg2,▆███▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁
metrics/mAP50(B),▁▄▄▅▅▆▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇███▇█▇████████
metrics/mAP50-95(B),▁▂▃▅▅▆▆▆▇▇▇▇▇▇▇█▇█▇█████████████████████
metrics/precision(B),▁▅▆▆▆▆▆▇▇▆▇▇▇▇▆▇█▇▇▇▇▇█▇▇▇▇▇▇█▇▇▇▇▇█▇▇▇█
metrics/recall(B),▁▂▆▆▆▇▇▆▇▇▇▇▇▇█▇█▇█▇▇▇▇█▇█▇▇███▇█▇█▇▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [6]:
import torch
import ultralytics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ultralytics.YOLO('yolov8s.yaml').to(device)

res = model.train(data='data.yaml', pretrained=True, epochs=200, batch=16, cache=False, project="mask_detection", name="yolo8s")

New https://pypi.org/project/ultralytics/8.3.94 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=mask_detection, name=yolo8s4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


train: Scanning /home/andrey/projects/ai_project/train/labels.cache... 683 images, 0 backgrounds, 0 corrupt: 100%|██████████| 683/683 [00:00<?, ?it/s]
val: Scanning /home/andrey/projects/ai_project/valid/labels.cache... 170 images, 0 backgrounds, 0 corrupt: 100%|██████████| 170/170 [00:00<?, ?it/s]


Plotting labels to mask_detection/yolo8s4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to mask_detection/yolo8s4
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


libpng warning: iCCP: Not recognizing known sRGB profile that has been edited:  33%|███▎      | 14/43 [00:02<00:05,  5.46it/s]
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited:  86%|████████▌ | 37/43 [00:06<00:00,  6.34it/s]
      1/200      3.79G       4.07      4.554      4.134         55        640: 100%|██████████| 43/43 [00:07<00:00,  5.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:00<00:00,  9.31it/s]

...

                   all        170        779      0.846       0.78      0.828      0.556



200 epochs completed in 0.428 hours.
Optimizer stripped from mask_detection/yolo8s4/weights/last.pt, 22.5MB
Optimizer stripped from mask_detection/yolo8s4/weights/best.pt, 22.5MB

Validating mask_detection/yolo8s4/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7930MiB)
YOLOv8s summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  4.78it/s]


                   all        170        779      0.885      0.781      0.846      0.566
             with_mask        157        640      0.945       0.88      0.947      0.652
          without_mask         54        116      0.819      0.741      0.791       0.52
 mask_weared_incorrect         19         23      0.892      0.722        0.8      0.527
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to mask_detection/yolo8s4


lr/pg0,█████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg1,▃████▇▇▇▇▇▆▆▆▆▆▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
lr/pg2,▆███▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
metrics/mAP50(B),▁▄▃▅▅▆▆▆▆▆▆▇▇▇█▇▇▇█▇█▇██▇▇██████████████
metrics/mAP50-95(B),▁▂▃▄▄▆▆▇▆▆▆▆▇█▇▇▇▇▇▇█▇▇█▇███████████████
metrics/precision(B),▁▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██▇▇▇█████▇█▇▇█▇████████
metrics/recall(B),▁▃▃▅▅▆▇▇▇▇▇▇▇████▇████▇████▇██▇████▇████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▆▅▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁
